In [24]:
def get_months_between(start_date, end_date):
    # Convertim les dates passades com a strings a objectes datetime
    start = datetime.strptime(start_date, '%d/%m/%Y')
    end = datetime.strptime(end_date, '%d/%m/%Y')

    # Ens assegurem que la data d'inici sigui anterior o igual a la data final
    if start > end:
        start, end = end, start

    # Incrementem un mes la data final
    end_month = (end.month % 12) + 1
    end_year = end.year + (end.month // 12)
    end = datetime(end_year, end_month, 1)

    # Generem la llista de mesos
    months = []
    current = start.replace(day=1)
    while current <= end:
        months.append(current.strftime('%d/%m/%Y 00:00:00'))
        # Incrementem al mes següent
        next_month = (current.month % 12) + 1
        next_year = current.year + (current.month // 12)
        current = current.replace(year=next_year, month=next_month)

    return months

In [26]:
get_months_between('01/01/2024', '01/03/2024')

['01/01/2024 00:00:00',
 '01/02/2024 00:00:00',
 '01/03/2024 00:00:00',
 '01/04/2024 00:00:00']

# 1. Setup Driver

In [2]:
import sys
sys.path.append("../")
from src.driver import setup_driver
driver = setup_driver()

# 2. Access to KINEO

In [4]:
from src.login import login
from private.config import USER_KEY, PASS_KEY

# login KINEO
login(driver,
      url = 'https://aforadores.mitma.es/contadorestraficofomento/Login.aspx',
      username = USER_KEY, 
      password = PASS_KEY)

In [5]:
# from src.dropdown import get_all_combinations
# driver.get('https://aforadores.mitma.es/contadorestraficofomento/InformeVolumenTraficoAgrupadoAforo.aspx')
# get_all_combinations(driver, 'ctl00_ContentPlaceHolderDatos_CbDemarcacion_B-1', 'ctl00_ContentPlaceHolderDatos_CbEtd_B-1', '../../input/values.py')

# 3. Make the requests and download data

In [7]:
from src.download import download_data
from input.values import VALUES_gerard

'''
# Por minutos 
choice = 'por_minutos'
desglose = 1, 5, 10 o 15 (minuts)
clock = 'hour' (es baixa un Excel per a cada hora), 'day' (es baixa un Excel per a cada dia), 'month' (es baixa un Excel per a cada mes), '2_months' (es baixa un Excel per a cada 2 mesos) o None (es baixa un Excel per a l'interval indicat)

# Por horas
choice = 'por_horas'
desglose = None
clock = None (es baixa un Excel per a l'interval indicat) 
'''

for values in VALUES_gerard:
    download_data(driver,
                  demarcacion = values["demarcacion"],
                  etd = values["etd"],
                  fecha_inicio = '01/06/2022',
                  fecha_fin = '01/06/2024',
                  choice = 'por_minutos', 
                  desglose = 1, 
                  clock = '2_months'
                 )

Exit driver

In [9]:
driver.quit()

# 4. Move files to desired directory

In [11]:
from src.directory import move_to_directory
from private.config import DOWNLOADS_PATH, L_PATH, G_PATH

move_to_directory(origin = DOWNLOADS_PATH,
                  destination = G_PATH)